In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm, tqdm_notebook,tqdm_pandas
import itertools
import re
import json

In [2]:
#stopword
sw = list(pd.read_excel("stopword(cp949).xlsx",encoding = 'cp949')['불용어']) #불용어 불러오기

#개별 불용어 
with open('custom_sw.json') as load_file:
    custom_sw = json.load(load_file)

In [39]:
def count_word(df, tag, keyword, stopword, custom_sw, save_name="", start_year=2010, end_year=2020, gap=1, month=False, months="", save=True):
    '''단어가 등장한 횟수를 카운팅하고, 각 연도별 순위변동을 표시
    month변수를 사용하여 특정 월별로 데이터 한정 가능
    엑셀로 바로 출력해서 사용하기 위해 multi_column으로 출력

    Parameters
    ----------
        df (dataFrame): DataFrame
        tag (string) : 품사지정
        costom_sw (dict) : 카운팅에서 제외할 불용어사전
        save_name : 저장파일이름
        
        start_year (int) : 순위변동 확인시 시작할 연도위치
        end_year (int) : 순위변동 확인시 마지막 연도위치 
        gap (int): 연도별 기간차이 지정
        month (bool) : 특정월 추출여부
        months (list) : 추출할 월list
        save : 파일저장여부
       
    Returns
    -------
    vertical_df : DataFrame
    save = True일때  './output/연도별카운트/' 에 저장

    '''
    
    df = df.fillna("")
    df['Noun'] = df['Noun'].apply(lambda x : re.sub("[\[\]' ]","",x).split(','))
    df['Adjective'] = df['Adjective'].apply(lambda x : re.sub("[\[\]' ]","",x).split(','))
    
    #stopword
    stopword = stopword +[keyword]
    try :
        stopword = stopword +custom_sw[keyword]
        stopword = list(set(stopword))
    except:
        pass
    
    main_df = pd.DataFrame()

    if month == True:
        df = df[df.month.isin(months)]
    for s_year in range(2010, 2020):
        word_year_df = df[df['year'] == s_year]

        target_df = list(itertools.chain(*word_year_df[tag]))
        counted = pd.DataFrame(
            pd.Series(target_df).value_counts()).reset_index()
        counted.columns = ['name', 'value']

        drop_row = counted[counted['name'].isin(stopword)].index
        counted.drop(drop_row, inplace=True)
        counted = counted.head(15)

        counted['year'] = s_year
        counted['rank'] = counted.value.rank(ascending=False, method='dense')
        main_df = pd.concat([main_df, counted])

    # gap
    main_df['gap'] = 0
    years = []
    for one_year in range(start_year, end_year+1, gap):
        years.append(one_year)
        names = main_df.loc[(main_df['year'] == one_year), "name"].values
        for name in names:
            try:
                new_rank = main_df.loc[(main_df['year'] == one_year) & (
                    main_df['name'] == name), 'rank'].values[0]
                old_rank = main_df.loc[(
                    main_df['year'] == one_year-gap) & (main_df['name'] == name), 'rank'].values[0]
                main_df.loc[(main_df['year'] == one_year) & (
                    main_df['name'] == name), 'gap'] = old_rank-new_rank
            except:
                main_df.loc[(main_df['year'] == one_year) & (
                    main_df['name'] == name), 'gap'] = "NEW"
    main_df = main_df[main_df['year'].isin(years)]
    main_df.drop('rank', axis=1, inplace=True)
    main_df.reset_index(drop = True, inplace = True)
    #vertical
    vertical_df = pd.DataFrame()
    for year in main_df.year.unique():
        sperated = main_df[main_df.year == year].reset_index(drop = True)
        vertical_df = pd.concat([vertical_df,sperated],axis = 1)
    
    #col_name_change 
    iteration = len(vertical_df.columns)//4
    col_names = ["키워드","검색량","year","순위"] * iteration
    vertical_df.columns = col_names
    
    #mult_cols
    years = vertical_df.year.iloc[0].values
    years_cols =[]
    for year in years:
        years_cols.extend([year]*3)
        
    vertical_df =vertical_df.drop('year',axis =1)
    origin_col = vertical_df.columns.values

    vertical_df.columns =[years_cols,origin_col]
    
    if save == True:
        vertical_df.to_csv('./output/연도별카운트/{save_name}.csv'.format(
            save_name=save_name), encoding='cp949', index='False')

    return vertical_df

In [40]:
path = './output/token_통합/'
file_list = os.listdir(path)

for file in tqdm_notebook(file_list):
    all_df = pd.read_csv(path+file)
    keyword = file.split('_')[0].split('+')[0].replace(' ','')
    count_word(all_df, 'Noun', keyword, sw,custom_sw, start_year=2011, end_year=2019, gap = 2, save_name = f'{keyword}_2년_gap명사')

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


### sample

In [10]:
path = './output/token_통합/'
file_list = os.listdir(path)
file = '하광정_16781.csv' # sample
all_df = pd.read_csv(path+file)
keyword = file.split('_')[0].split('+')[0].replace(' ','')

In [11]:
vertical_df = count_word(all_df, 'Noun', keyword, sw,custom_sw, gap = 2, save = False,start_year=2011, end_year=2019, )

ValueError: all arrays must be same length

In [232]:
vertical_df

2011             2013             2015             2017             2019  \
     키워드   검색량   순위   키워드   검색량   순위   키워드   검색량   순위   키워드   검색량   순위   키워드   
0     펜션  1158  NEW    펜션  1306    0  해수욕장  2434    1  해수욕장  3301    0    서핑   
1   해수욕장   973  NEW  해수욕장  1250    0    속초  1932    1    속초  2419    0  해수욕장   
2     속초   680  NEW    속초   896    0    펜션  1266   -2    펜션  2080    0    펜션   
3     바위   536  NEW    등대   767    4    일출  1248    2    서핑  1813    3    속초   
4     동해   475  NEW    동해   723    0    동해  1195    0    동해  1476    0    맛집   
5     계곡   467  NEW    일출   715    4    등대  1164   -2    여름  1425    4    카페   
6     마을   455  NEW    계곡   685   -1    서핑  1056  NEW   전망대  1310  NEW    강릉   
7     등대   392  NEW   낙산사   603    3    파도  1047    7    등대  1288   -2    피비   
8    설악산   385  NEW    바위   553   -5    바위   956    0    파도  1265   -1    숙소   
9     일출   371  NEW    마을   534   -3    여름   866    3   막국수  1215  NEW   막국수   
10   낙산사   348  NEW    강릉   517    1   낙산사   857   -3    강릉  1212    1    여름   
11    강릉   345  NEW   하조대   500  NEW    강릉   848   -1    맛집  1164  NEW    비치   
12   주문진   296  NEW    여름   483  NEW    계곡   793   -6    서울  1063  NEW   전망대   
13    코스   278  NEW    서울   472  NEW    코스   787  NEW  고속도로  1052  NEW    파도   
14    가족   263  NEW    파도   423  NEW    가족   742  NEW    일출  1034  -11    동해   

               
     검색량   순위  
0   3522    3  
1   3492   -1  
2   3407    0  
3   2917   -2  
4   2691    7  
5   2572  NEW  
6   2199    4  
7   2027  NEW  
8   2025  NEW  
9   1971    0  
10  1931   -5  
11  1865  NEW  
12  1648   -6  
13  1596   -5  
14  1590  -10

### 이하작업용

In [24]:
path = './output/token_통합/'
file_list = os.listdir(path)
file = '하광정_16781.csv' # sample
all_df = pd.read_csv(path+file)
tag = 'Noun'
keyword = file.split('_')[0].split('+')[0].replace(' ','')
main_df = pd.DataFrame()

In [25]:
all_df = all_df.fillna("")
all_df['Noun'] = all_df['Noun'].apply(lambda x : re.sub("[\[\]' ]","",x).split(','))
all_df['Adjective'] = all_df['Adjective'].apply(lambda x : re.sub("[\[\]' ]","",x).split(','))

In [26]:
#stopword
sw.append(keyword)
try :
    stopword = sw +custom_sw[keyword]
    stopword = list(set(stopword))
except:
    pass

In [27]:
for s_year in range(2010, 2020):
    word_year_df = all_df[all_df['year'] == s_year]

    target_df = list(itertools.chain(*word_year_df[tag]))
    counted = pd.DataFrame(
        pd.Series(target_df).value_counts()).reset_index()
    counted.columns = ['name', 'value']

    drop_row = counted[counted['name'].isin(stopword)].index
    counted.drop(drop_row, inplace=True)
    counted = counted.head(15)

    counted['year'] = s_year
    counted['rank'] = counted.value.rank(ascending=False,method='dense')
    main_df = pd.concat([main_df, counted])

In [28]:
main_df[main_df.name == '양양']

,name,value,year,rank


In [29]:
start_year = 2011
end_year =2019
gap =  2

In [30]:
main_df['gap'] = 0
years = []
for one_year in range(start_year, end_year, gap):
    years.append(one_year)
    names = main_df.loc[(main_df['year'] == one_year), "name"].values
    for name in names:
        try:
            new_rank = main_df.loc[(main_df['year'] == one_year) & (
                main_df['name'] == name), 'rank'].values[0]
            old_rank = main_df.loc[(
                main_df['year'] == one_year-gap) & (main_df['name'] == name), 'rank'].values[0]
            main_df.loc[(main_df['year'] == one_year) & (
                main_df['name'] == name), 'gap'] = old_rank-new_rank
        except:
            main_df.loc[(main_df['year'] == one_year) & (
                main_df['name'] == name), 'gap'] = "NEW"

In [31]:
main_df = main_df[main_df['year'].isin(years)]
main_df.drop('rank', axis=1, inplace=True)
main_df.reset_index(drop = True, inplace = True)
#vertical
vertical_df = pd.DataFrame()
for year in main_df.year.unique():
    sperated = main_df[main_df.year == year].reset_index(drop = True)
    vertical_df = pd.concat([vertical_df,sperated],axis = 1)

In [33]:
vertical_df

,name,value,year,gap,name,value,year,gap,name,value,year,gap,name,value,year,gap
0,펜션,1158,2011,NEW,펜션,1306,2013,0,해수욕장,2434,2015,1,해수욕장,3301,2017,0
1,해수욕장,973,2011,NEW,해수욕장,1250,2013,0,속초,1932,2015,1,속초,2419,2017,0
2,속초,680,2011,NEW,속초,896,2013,0,펜션,1266,2015,-2,펜션,2080,2017,0
3,바위,536,2011,NEW,등대,767,2013,4,일출,1248,2015,2,서핑,1813,2017,3
4,동해,475,2011,NEW,동해,723,2013,0,동해,1195,2015,0,동해,1476,2017,0
5,계곡,467,2011,NEW,일출,715,2013,4,등대,1164,2015,-2,여름,1425,2017,4
6,마을,455,2011,NEW,계곡,685,2013,-1,서핑,1056,2015,NEW,전망대,1310,2017,NEW
7,등대,392,2011,NEW,낙산사,603,2013,3,파도,1047,2015,7,등대,1288,2017,-2
8,설악산,385,2011,NEW,바위,553,2013,-5,바위,956,2015,0,파도,1265,2017,-1
9,일출,371,2011,NEW,마을,534,2013,-3,여름,866,2015,3,막국수,1215,2017,NEW


In [36]:
years_cols

[2011,
 2011,
 2011,
 2011,
 2013,
 2013,
 2013,
 2013,
 2015,
 2015,
 2015,
 2015,
 2017,
 2017,
 2017,
 2017]

In [37]:
vertical_df

,키워드,검색량,순위,키워드,검색량,순위,키워드,검색량,순위,키워드,검색량,순위
0,펜션,1158,NEW,펜션,1306,0,해수욕장,2434,1,해수욕장,3301,0
1,해수욕장,973,NEW,해수욕장,1250,0,속초,1932,1,속초,2419,0
2,속초,680,NEW,속초,896,0,펜션,1266,-2,펜션,2080,0
3,바위,536,NEW,등대,767,4,일출,1248,2,서핑,1813,3
4,동해,475,NEW,동해,723,0,동해,1195,0,동해,1476,0
5,계곡,467,NEW,일출,715,4,등대,1164,-2,여름,1425,4
6,마을,455,NEW,계곡,685,-1,서핑,1056,NEW,전망대,1310,NEW
7,등대,392,NEW,낙산사,603,3,파도,1047,7,등대,1288,-2
8,설악산,385,NEW,바위,553,-5,바위,956,0,파도,1265,-1
9,일출,371,NEW,마을,534,-3,여름,866,3,막국수,1215,NEW


In [38]:
#col_name_change 
iteration = len(vertical_df.columns)//4
col_names = ["키워드","검색량","year","순위"] * iteration
vertical_df.columns = col_names

#mult_cols
years = vertical_df.year.iloc[0].values
years_cols =[]
for year in years:
    years_cols.extend([year]*3)

vertical_df =vertical_df.drop('year',axis =1)
origin_col = vertical_df.columns.values

vertical_df.columns =[years_cols,origin_col]

In [101]:
counted.head(10)

,name,value
0,강릉,2931
1,서울,1612
2,마을,1574
3,강원,1437
4,계곡,1345
5,한국,1025
6,해수욕장,948
7,동해,907
8,체험,858
9,문화,793


In [84]:
stopword = ['강릉','강릉시','강원']

In [90]:
drop_row = counted[counted['name'].isin(stopword)].index
counted.drop(drop_row, inplace=True)
counted = counted.head(15)

counted['year'] = s_year
counted['rank'] = counted.value.rank(ascending=False)
# main_df = pd.concat([main_df, counted])

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


NameError: name 'main_df' is not defined

In [91]:
counted

,name,value,year,rank
1,마을,1402,2010,1.0
2,커피,1249,2010,2.0
3,평창,1198,2010,3.0
4,서울,1056,2010,4.0
5,코스,1010,2010,5.0
7,이야기,998,2010,6.0
8,펜션,978,2010,7.0
9,한국,894,2010,8.0
10,해수욕장,880,2010,9.0
11,동해,863,2010,10.0
